# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get data from `.csv` files and then preprocess data

In [2]:
# next Get data from .csv files and then preprocess data
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    # Read CSVs
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_fr  = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp  = pd.read_csv("data/movie_reviews_sp.csv")
    
    # Map columns to standard names for each dataframe
    df_eng = df_eng.rename(columns={
        df_eng.columns[0]: "Title",
        df_eng.columns[1]: "Year",
        df_eng.columns[2]: "Synopsis",
        df_eng.columns[3]: "Review"
    })
    df_fr = df_fr.rename(columns={
        df_fr.columns[0]: "Title",
        df_fr.columns[1]: "Year",
        df_fr.columns[2]: "Synopsis",
        df_fr.columns[3]: "Review"
    })
    df_sp = df_sp.rename(columns={
        df_sp.columns[0]: "Title",
        df_sp.columns[1]: "Year",
        df_sp.columns[2]: "Synopsis",
        df_sp.columns[3]: "Review"
    })
    
    # Add Original Language column
    df_eng["Original Language"] = "en"
    df_fr["Original Language"]  = "fr"
    df_sp["Original Language"]  = "sp"
    
    # Concatenate all dataframes
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    
    return df

df = preprocess_data()


In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",fr
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",sp
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en
22,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",sp
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",fr
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",fr
17,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",fr
10,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",fr


### What was done:

Step 1: Fetch and Preprocess Data

In this step, we read movie review data from three separate CSV files — one for each language: English, French, and Spanish. Each CSV had slightly different column names, so we standardized them to:

["Title", "Year", "Synopsis", "Review"]


We also added a new column "Original Language" to indicate the language of each review before translation ("en", "fr", "sp").

Finally, we concatenated all three dataframes into a single dataframe df containing all 30 movies, ready for further processing.

Key actions performed:

Loaded CSVs using pd.read_csv().

Renamed columns for consistency.

Added "Original Language" to preserve source language information.

Combined all rows into one dataframe using pd.concat().

Resulting dataframe preview:

| Title           | Year        | Synopsis                    | Review                                 | Original Language |
| --------------- | ----------- | --------------------------- | -------------------------------------- | ----------------- |
| The Dark Knight | 2008        | Batman (Christian Bale)...  | "The Dark Knight is a thrilling..."    | en                |
| Amélie          | 2001        | Cette comédie romantique... | "Amélie est un film absolument..."     | fr                |
| Águila Roja     | (2009-2016) | Esta serie de televisión... | "Águila Roja es una serie aburrida..." | sp                |


This unified dataframe is now ready for translation and sentiment analysis.

### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
from tqdm import tqdm
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# ===============================
# TODO 2: Load translation models and tokenizers
# ===============================

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# ===============================
# TODO 3: Complete the translation function
# ===============================

def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    if pd.isna(text) or text.strip() == "":
        return ""
    
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    
    # generate the translation using the model
    outputs = model.generate(**inputs, max_new_tokens=512)
    
    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# ===============================
# TODO 4: Filter and translate dataframe columns, then update dataframe
# ===============================

# filter reviews in French and translate to English
fr_reviews = df.loc[df["Original Language"] == "fr", "Review"]
fr_reviews_en = [translate(x, fr_en_model, fr_en_tokenizer) for x in tqdm(fr_reviews, desc="Translating French Reviews")]

# filter synopsis in French and translate to English
fr_synopsis = df.loc[df["Original Language"] == "fr", "Synopsis"]
fr_synopsis_en = [translate(x, fr_en_model, fr_en_tokenizer) for x in tqdm(fr_synopsis, desc="Translating French Synopsis")]

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df["Original Language"] == "sp", "Review"]
es_reviews_en = [translate(x, es_en_model, es_en_tokenizer) for x in tqdm(es_reviews, desc="Translating Spanish Reviews")]

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df["Original Language"] == "sp", "Synopsis"]
es_synopsis_en = [translate(x, es_en_model, es_en_tokenizer) for x in tqdm(es_synopsis, desc="Translating Spanish Synopsis")]

# update dataframe with translated text
# overwrite the existing data with English translations
df.loc[df["Original Language"] == "fr", "Review"] = fr_reviews_en
df.loc[df["Original Language"] == "fr", "Synopsis"] = fr_synopsis_en
df.loc[df["Original Language"] == "sp", "Review"] = es_reviews_en
df.loc[df["Original Language"] == "sp", "Synopsis"] = es_synopsis_en


Translating Spanish Synopsis: 100%|██████████| 10/10 [00:45<00:00,  4.55s/it]


In [6]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",fr
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp


### what was done 

Step 2: Translate French and Spanish Reviews to English

In this step, we translated all non-English movie reviews and synopses into English so that sentiment analysis can be performed consistently across all movies.

Key actions performed:

Load pre-trained translation models and tokenizers from HuggingFace:

French → English: Helsinki-NLP/opus-mt-fr-en

Spanish → English: Helsinki-NLP/opus-mt-es-en

fr_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
fr_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
es_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-es-en")
es_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")


Created a translation function translate(text, model, tokenizer) that:

Encodes text using the tokenizer.

Generates translation with the model.

Decodes the output into readable English text.

Filtered French and Spanish reviews and synopses from the dataframe, applied the translation function, and updated the dataframe:

Overwrote the original "Review" and "Synopsis" columns with English translations.

Added a progress bar using tqdm to track translation progress for each language.

fr_reviews_en = [translate(x, fr_en_model, fr_en_tokenizer) for x in tqdm(fr_reviews, desc="Translating French Reviews")]
es_reviews_en = [translate(x, es_en_model, es_en_tokenizer) for x in tqdm(es_reviews, desc="Translating Spanish Reviews")]


Resulting dataframe preview: All reviews and synopses are now in English, while "Original Language" still preserves the source language ("en", "fr", "sp").

| Title             | Year | Synopsis                    | Review                                              | Original Language |
| ----------------- | ---- | --------------------------- | --------------------------------------------------- | ----------------- |
| Blade Runner 2049 | 2017 | Officer K (Ryan Gosling)... | "Boring and too long. Nothing like the original..." | en                |
| Amores perros     | 2000 | Three stories intertwine... | "Amores dogs is an intense and moving film..."      | sp                |
| Babylon A.D.      | 2008 | In the distant future...    | "This film is a complete mess. The characters..."   | fr                |


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
from transformers import pipeline

# ===============================
# TODO 5: Load sentiment analysis model
# ===============================
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# ===============================
# TODO 6: Complete the function to analyze sentiment
# ===============================
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    returns 'Positive' or 'Negative'
    """
    if pd.isna(text) or text.strip() == "":
        return "Neutral"  # optional handling for empty reviews
    
    result = classifier(text)[0]  # returns dict with 'label' and 'score'
    label = result['label']
    
    # convert HuggingFace label to Positive/Negative
    if label.upper() == "POSITIVE":
        return "Positive"
    else:
        return "Negative"

# ===============================
# Apply sentiment analysis to the dataframe
# ===============================
df["Sentiment"] = df["Review"].apply(lambda x: analyze_sentiment(x, sentiment_classifier))


In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column

from tqdm import tqdm

tqdm.pandas(desc="Analyzing Sentiment")  # enable progress bar for apply

df["Sentiment"] = df["Review"].progress_apply(lambda x: analyze_sentiment(x, sentiment_classifier))


Analyzing Sentiment: 100%|██████████| 30/30 [00:06<00:00,  4.92it/s]


In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",fr,Negative
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp,Negative
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp,Negative
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en,Positive
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp,Negative
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp,Positive
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr,Positive
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp,Negative
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en,Negative
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",fr,Positive


### what was done 

Step 3: Sentiment Analysis of Movie Reviews

After translating all reviews into English, we analyzed the sentiment of each review using a pre-trained HuggingFace model.

Key actions performed:

Load a sentiment analysis model:

Model used: distilbert-base-uncased-finetuned-sst-2-english

Pipeline task: "sentiment-analysis"

from transformers import pipeline
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)


Created a helper function analyze_sentiment(text, classifier) that:

Handles empty or missing reviews by returning "Neutral" (optional).

Uses the classifier to predict sentiment (POSITIVE/NEGATIVE).

Converts the HuggingFace labels into "Positive" or "Negative".

def analyze_sentiment(text, classifier):
    if pd.isna(text) or text.strip() == "":
        return "Neutral"
    result = classifier(text)[0]
    return "Positive" if result['label'].upper() == "POSITIVE" else "Negative"


Applied the sentiment analysis function to all reviews:

Used tqdm to show a progress bar while processing the dataframe.

Stored results in a new column "Sentiment".

from tqdm import tqdm
tqdm.pandas(desc="Analyzing Sentiment")
df["Sentiment"] = df["Review"].progress_apply(lambda x: analyze_sentiment(x, sentiment_classifier))


Resulting dataframe preview:
| Title                       | Year | Synopsis                                        | Review                                                     | Original Language | Sentiment |
| --------------------------- | ---- | ----------------------------------------------- | ---------------------------------------------------------- | ----------------- | --------- |
| Astérix aux Jeux Olympiques | 2008 | In this film adaptation...                      | "This film is a complete disappointment..."                | fr                | Negative  |
| El Incidente                | 2014 | In this Mexican horror film...                  | "The Incident is a boring and frightless film..."          | sp                | Negative  |
| Forrest Gump                | 1994 | Forrest Gump (Tom Hanks) is a simple man...     | "Forrest Gump is a heartwarming and inspirational film..." | en                | Positive  |
| Les Choristes               | 2004 | This film tells the story of a music teacher... | "The Choristes are a beautiful film that will..."          | fr                | Positive  |


Outcome: Every movie review now has an associated sentiment, allowing us to compare positivity/negativity across languages and genres.

In [11]:
import os

# create the folder if it doesn't exist
os.makedirs("result", exist_ok=True)

# now export the dataframe
df.to_csv("result/reviews_with_sentiment.csv", index=False)
